In [16]:
import pandas as pd
import numpy as np
import collections
from collections import Counter

Описание задачи

Небольшой интернет-магазин попросил вас добавить ранжирование товаров в блок "Смотрели ранее" - в нем теперь надо показывать не последние просмотренные пользователем товары, а те товары из просмотренных, которые он наиболее вероятно купит. Качество вашего решения будет оцениваться по количеству покупок в сравнении с прошлым решением в ходе А/В теста, т.к. по доходу от продаж статзначимость будет достигаться дольше из-за разброса цен. Таким образом, ничего заранее не зная про корреляцию оффлайновых и онлайновых метрик качества, в начале проекта вы можете лишь постараться оптимизировать recall@k и precision@k.

Это задание посвящено построению простых бейзлайнов для этой задачи: ранжирование просмотренных товаров по частоте просмотров и по частоте покупок. Эти бейзлайны, с одной стороны, могут помочь вам грубо оценить возможный эффект от ранжирования товаров в блоке - например, чтобы вписать какие-то числа в коммерческое предложение заказчику, а с другой стороны, могут оказаться самым хорошим вариантом, если данных очень мало (недостаточно для обучения даже простых моделей).

Входные данные

Вам дается две выборки с пользовательскими сессиями - id-шниками просмотренных и id-шниками купленных товаров. Одна выборка будет использоваться для обучения (оценки популярностей товаров), а другая - для теста.

В файлах записаны сессии по одной в каждой строке. Формат сессии: id просмотренных товаров через , затем идёт ; после чего следуют id купленных товаров (если такие имеются), разделённые запятой. Например, 1,2,3,4; или 1,2,3,4;5,6.

Гарантируется, что среди id купленных товаров все различные.

Важно:

- Сессии, в которых пользователь ничего не купил, исключаем из оценки качества.
- Если товар не встречался в обучающей выборке, его популярность равна 0.
- Рекомендуем разные товары. И их число должно быть не больше, чем количество различных просмотренных пользователем товаров.
- Рекомендаций всегда не больше, чем минимум из двух чисел: количество просмотренных пользователем товаров и k в recall@k / precision@k.

Задание

1) На обучении постройте частоты появления id в просмотренных и в купленных (id может несколько раз появляться в просмотренных, все появления надо учитывать.

2) Реализуйте два алгоритма рекомендаций:
- сортировка просмотренных id по популярности (частота появления в просмотренных),
- сортировка просмотренных id по покупаемости (частота появления в покупках).

3) Для данных алгоритмов выпишите через пробел AverageRecall@1, AveragePrecision@1, AverageRecall@5, AveragePrecision@5 на обучающей и тестовых выборках, округляя до 2 знака после запятой. Это будут ваши ответы в этом задании. Посмотрите, как они соотносятся друг с другом. Где качество получилось выше? Значимо ли это различие? Обратите внимание на различие качества на обучающей и тестовой выборке в случае рекомендаций по частотам покупки.

Если частота одинаковая, то сортировать нужно по возрастанию момента просмотра (чем раньше появился в просмотренных, тем больше приоритет)

Дополнительные вопросы

1) Обратите внимание, что при сортировке по покупаемости возникает много товаров с одинаковым рангом - это означает, что значение метрик будет зависеть от того, как мы будем сортировать товары с одинаковым рангом. Попробуйте убедиться, что при изменении сортировки таких товаров recall@k меняется. Подумайте, как оценить минимальное и максимальное значение recall@k в зависимости от правила сортировки.

2) Мы обучаемся и тестируемся на полных сессиях (в которых есть все просмотренные за сессию товары). Подумайте, почему полученная нами оценка качества рекомендаций в этом случае несколько завышена.


In [165]:
with open('train.txt', 'r') as f:
    train = f.read().splitlines()
with open('test.txt', 'r') as f:
    test = f.read().splitlines()

In [166]:
train_split = []

for tr in train:
    visit, perchases = tr.split(';')
    visit = list(map(int, visit.split(',')))
    if perchases != '':
        perchase = list(map(int,perchases.split(',')))
    else:
        perchase = []
    train_split.append([visit, perchase])


[]

In [167]:
test_split = []
for tr in test:
    visit, perchases = tr.split(';')
    visit = list(map(int, visit.split(',')))
    if perchases != '':
        perchase = list(map(int,perchases.split(',')))
    else:
        perchase = []
    test_split.append([visit, perchase])

len(test_split)

50000

In [168]:
visit_train_l = [row[0] for row in train_split]
visit_train_np = np.array( [id_n for visit in visit_train_l for id_n in visit] )

visit_train_unique = np.transpose(np.unique(visit_train_np, return_counts=True))
visit_train_unique

array([[     0,      6],
       [     1,      6],
       [     2,      9],
       ...,
       [102804,      1],
       [102805,      1],
       [102806,      1]], dtype=int64)

In [140]:
visit_train_l[-5]

[32291, 60520, 32291, 38220]

In [169]:
perchase_train_l = [row[1] for row in train_split]
perchase_train_np = np.array( [id_n for perchase in perchase_train_l for id_n in perchase] )

perchase_train_unique = np.transpose(np.unique(perchase_train_np, return_counts=True))
perchase_train_unique

array([[     5,      1],
       [     6,      2],
       [     7,      2],
       ...,
       [102417,      1],
       [102462,      1],
       [102646,      1]], dtype=int64)

In [101]:
visit_train_unique.sort()

In [175]:
visit_train_unique = visit_train_unique[visit_train_unique[:,1].argsort()][::-1]
perchase_train_unique = perchase_train_unique[perchase_train_unique[:,1].argsort()][::-1]


In [171]:
visit_train_unique_sort

array([[    73,    677],
       [   158,    641],
       [   204,    396],
       ...,
       [ 73827,      1],
       [ 73828,      1],
       [102806,      1]], dtype=int64)

In [128]:
perchase_train_unique_sort

array([[  158,    14],
       [  204,    12],
       [   73,    11],
       ...,
       [38189,     1],
       [38177,     1],
       [    5,     1]], dtype=int64)

In [172]:
def prec_rec_metrics(session, reccomendations, k):
    purchase = 0
    for ind in reccomendations:
        if ind in session:
            purchase += 1 
    precision = purchase / k
    recall = purchase / len(session)
    return(precision, recall)

In [176]:
# По просмотрам
prec_at_1_tr_l, rec_at_1_tr_l = [], []
prec_at_5_tr_l, rec_at_5_tr_l = [], []
k1, k5 = 1, 5
for i, sess_perchase in enumerate(perchase_train_l):
    # Пропускаем пустые
    if sess_perchase == []: continue
    
    # visit ids, где perchare != ''
    sess_l = visit_train_l[i]

    # sorted looks ids indices in sess_train_l_cnt array
    # sort in accordance with looks counts
    l_ind_sess = []
    for j in range(len(sess_l)):
        l_ind_sess.append(np.where(visit_train_unique[:,0] == sess_l[j])[0][0])
    l_ind_sess_sorted = np.unique(l_ind_sess)
    
    num_of_recs_k1 = min(k1, len(sess_l))
    if num_of_recs_k1 == 0: continue
    recs_k1 = visit_train_unique[l_ind_sess_sorted[:num_of_recs_k1],0]
    
    num_of_recs_k5 = min(k5, len(sess_l))
    if num_of_recs_k5 == 0: continue
    recs_k5 = visit_train_unique[l_ind_sess_sorted[:num_of_recs_k5],0]
    
    prec_1, rec_1 = prec_rec_metrics(sess_perchase, recs_k1, k1)
    prec_at_1_tr_l.append(prec_1)
    rec_at_1_tr_l.append(rec_1)
    
    prec_5, rec_5 = prec_rec_metrics(sess_perchase, recs_k5, k5)
    prec_at_5_tr_l.append(prec_5)
    rec_at_5_tr_l.append(rec_5)

In [177]:
avg_prec_at_1_tr_l = np.mean(prec_at_1_tr_l)
avg_rec_at_1_tr_l = np.mean(rec_at_1_tr_l)
avg_prec_at_5_tr_l = np.mean(prec_at_5_tr_l)
avg_rec_at_5_tr_l = np.mean(rec_at_5_tr_l)
avg_rec_at_1_tr_l

0.4383013528107763

In [178]:
with open('ans1.txt', 'w') as f:
    r1 = '%.2f' % round(avg_rec_at_1_tr_l, 2)
    p1 = '%.2f' % round(avg_prec_at_1_tr_l, 2)
    r5 = '%.2f' % round(avg_rec_at_5_tr_l, 2)
    p5 = '%.2f' % round(avg_prec_at_5_tr_l, 2)
    ans1 = ' '.join([r1, p1, r5, p5])
    print('Answer 1:', ans1)
    f.write(ans1)

Answer 1: 0.44 0.51 0.83 0.21


In [183]:
prec_at_1_tr_l, rec_at_1_tr_l = [], []
prec_at_5_tr_l, rec_at_5_tr_l = [], []
k1, k5 = 1, 5
for i, sess_perchase in enumerate(perchase_train_l):
    # Пропускаем пустые
    if sess_perchase == []: continue
    
    # visit ids, где perchare != ''
    sess_l = visit_train_l[i]

    # sorted looks ids indices in sess_train_l_cnt array
    # sort in accordance with looks counts
    l_ind_sess = []
    for j in range(len(sess_l)):
        if sess_l[j] not in perchase_train_unique[:,0]: continue
        l_ind_sess.append(np.where(perchase_train_unique[:,0] == sess_l[j])[0][0])
    l_ind_sess_sorted = np.unique(l_ind_sess)
    
    num_of_recs_k1 = min(k1, len(sess_l), len(l_ind_sess_sorted))
    if num_of_recs_k1 == 0: continue
    recs_k1 = perchase_train_unique[l_ind_sess_sorted[:num_of_recs_k1],0]
    
    num_of_recs_k5 = min(k5, len(sess_l), len(l_ind_sess_sorted))
    if num_of_recs_k5 == 0: continue
    recs_k5 = perchase_train_unique[l_ind_sess_sorted[:num_of_recs_k5],0]
    
    prec_1, rec_1 = prec_rec_metrics(sess_perchase, recs_k1, k1)
    prec_at_1_tr_l.append(prec_1)
    rec_at_1_tr_l.append(rec_1)
    
    prec_5, rec_5 = prec_rec_metrics(sess_perchase, recs_k5, k5)
    prec_at_5_tr_l.append(prec_5)
    rec_at_5_tr_l.append(rec_5)

In [184]:
avg_prec_at_1_tr_l = np.mean(prec_at_1_tr_l)
avg_rec_at_1_tr_l = np.mean(rec_at_1_tr_l)
avg_prec_at_5_tr_l = np.mean(prec_at_5_tr_l)
avg_rec_at_5_tr_l = np.mean(rec_at_5_tr_l)
avg_rec_at_1_tr_l

0.677355817184901

In [185]:
with open('ans2.txt', 'w') as f:
    r1 = '%.2f' % round(avg_rec_at_1_tr_l, 2)
    p1 = '%.2f' % round(avg_prec_at_1_tr_l, 2)
    r5 = '%.2f' % round(avg_rec_at_5_tr_l, 2)
    p5 = '%.2f' % round(avg_prec_at_5_tr_l, 2)
    ans2 = ' '.join([r1, p1, r5, p5])
    print('Answer 2:', ans2)
    f.write(ans2)

Answer 2: 0.68 0.79 0.93 0.25


In [210]:
visit_test_l = [row[0] for row in test_split]
visit_test_np = np.array( [id_n for visit in visit_test_l for id_n in visit] )

#visit_test_unique = np.transpose(np.unique(visit_test_np, return_counts=True))
#visit_test_unique

In [209]:
perchase_test_l = [row[1] for row in test_split]
perchase_test_np = np.array( [id_n for perchase in perchase_test_l for id_n in perchase] )

#perchase_test_unique = np.transpose(np.unique(perchase_test_np, return_counts=True))
#perchase_test_unique

In [188]:
visit_test_unique = visit_test_unique[visit_test_unique[:,1].argsort()][::-1]
perchase_test_unique = perchase_test_unique[perchase_test_unique[:,1].argsort()][::-1]

In [213]:
# По просмотрам
prec_at_1_tst_l, rec_at_1_tst_l = [], []
prec_at_5_tst_l, rec_at_5_tst_l = [], []
k1, k5 = 1, 5
for i, sess_perchase in enumerate(perchase_test_l):
    # Пропускаем пустые
    if sess_perchase == []: continue
    
    # visit ids, где perchare != ''
    sess_l = visit_test_l[i]

    # sorted looks ids indices in sess_train_l_cnt array
    # sort in accordance with looks counts
    l_ind_sess = []
    new_ids = []
    for j in range(len(sess_l)):
        if sess_l[j] not in visit_train_unique[:,0]:
            new_ids.append(sess_l[j])
            continue
        l_ind_sess.append(np.where(visit_train_unique[:,0] == sess_l[j])[0][0])
    l_ind_sess_sorted = np.unique(l_ind_sess)
    
    num_of_recs_k1 = min(k1, len(sess_l))
    if num_of_recs_k1 == 0: continue
    if l_ind_sess != []:
        recs_k1 = visit_train_unique[l_ind_sess_sorted[:num_of_recs_k1],0]
    else:
        recs_k1 = []
    recs_k1 = np.concatenate((np.array(recs_k1, dtype='int64'), np.unique(np.array(new_ids, dtype='int64'))))[:num_of_recs_k1]
    
    num_of_recs_k5 = min(k5, len(sess_l))
    if num_of_recs_k5 == 0: continue
    if l_ind_sess != []:
        recs_k5 = visit_train_unique[l_ind_sess_sorted[:num_of_recs_k5],0]
    else:
        recs_k5 = []
    recs_k5 = np.concatenate((np.array(recs_k5, dtype='int64'), np.unique(np.array(new_ids, dtype='int64'))))[:num_of_recs_k5]
    
    prec_1, rec_1 = prec_rec_metrics(sess_perchase, recs_k1, k1)
    prec_at_1_tst_l.append(prec_1)
    rec_at_1_tst_l.append(rec_1)
    
    prec_5, rec_5 = prec_rec_metrics(sess_perchase, recs_k5, k5)
    prec_at_5_tst_l.append(prec_5)
    rec_at_5_tst_l.append(rec_5)

In [214]:
avg_prec_at_1_tst_l = np.mean(prec_at_1_tst_l)
avg_rec_at_1_tst_l = np.mean(rec_at_1_tst_l)
avg_prec_at_5_tst_l = np.mean(prec_at_5_tst_l)
avg_rec_at_5_tst_l = np.mean(rec_at_5_tst_l)
avg_rec_at_1_te_l

0.03128694861300591

In [216]:
with open('ans3.txt', 'w') as f:
    r1 = '%.2f' % round(avg_rec_at_1_te_l, 2)
    p1 = '%.2f' % round(avg_prec_at_1_te_l, 2)
    r5 = '%.2f' % round(avg_rec_at_5_te_l, 2)
    p5 = '%.2f' % round(avg_prec_at_5_te_l, 2)
    ans3 = ' '.join([r1, p1, r5, p5])
    print('Answer 3:', ans3)
    f.write(ans3)

SyntaxError: invalid syntax (<ipython-input-216-c6657aead2f8>, line 8)